<a href="https://colab.research.google.com/github/reanqbow/SUMOGH/blob/master/FlexiStreet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **FlexiStreet**
## *Designing and Visualizing Proactive Street Usage in Post-pandemic MIT-Kendall Square*

Team Member: Clara He, Yubo Zhao, George Guida, Elyjana Roach

GitHub GIST: https://gist.github.com/reanqbow/58b1eda07464d1ab320f53d0382e3f48

@ MIT Media Lab City Science Group


# 1. Introduction
## 1.1 Abstract
**ADD NOTES!!!**
## 1.2 Keywords
Responsive Street, Data Visualization, Urban Policy, Sustainability, Lightweight Mobility, Traffic Simulation

## 1.3 Modules
*   Traffic Data: Along with our research on recent commute demand data, [SUMO](https://sumo.dlr.de/docs/index.html) is used to run an agent-based simulation that demonstrates vehicle travel info, pedestrian behaviors, spatial occupancies,and greenhouse gases emissions, in different scenarios.
*   Data Visualization: [Plotly](https://plotly.com/python/) and its integrated [Mapbox](https://plotly.com/python/mapbox-layers/) library are used to visualize the data in both 3D street view and statistical graphs.
*   FlexiStreet Analysis Module: Our own script that analyzes the outcomes based on four metrics.


# 2. System Prep

**ADD NOTES!!!**

## 2.1Installing Dependencies

In [1]:
# Installing SUMO
!pip install eclipse-sumo

     |████████████████████████████████| 50.6 MB 105 kB/s 


In [11]:
# Installing All Python Libraries Needed for Visualization and Data Processing
!pip install plotly==5.4.0

from google.colab import data_table

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET

data_table.enable_dataframe_formatter()
px.set_mapbox_access_token("pk.eyJ1IjoibGlhbXoiLCJhIjoiY2oxZnE5MHFhMDByeTJybzJrZjlvYTR4YyJ9.qwj-DfGjKw0dchnd5cNNDQ")
style="mapbox://styles/liamz/ckw3bsrr909vo14mybm880hq7"

## 2.2Set Up Environment Variables

In [3]:
%env SUMO_HOME=/usr/local/lib/python3.7/dist-packages/sumo
%env PYTHONPATH = 
! echo $SUMO_HOME

env: SUMO_HOME=/usr/local/lib/python3.7/dist-packages/sumo
env: PYTHONPATH=
/usr/local/lib/python3.7/dist-packages/sumo


In [4]:
import os, sys
sys.path.append('/usr/local/lib/python3.7/dist-packages/sumo/tools')

In [5]:
sys.path

['',
 '/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/usr/local/lib/python3.7/dist-packages/sumo/tools']

# 3. Street Simulation in SUMO

**ADD NOTES!!!**




In [ ]:
# Using SUMO as a Python Library
import sumolib

#!python /usr/local/lib/python3.7/dist-packages/sumo/tools/osmWebWizard.py

In [ ]:
#downloading the osm file of kendall
! wget -O kendall.osm "https://api.openstreetmap.org/api/0.6/map?bbox=-71.0952,42.3594,-71.0698,42.3690"

--2021-11-29 22:10:25--  https://api.openstreetmap.org/api/0.6/map?bbox=-71.0952,42.3594,-71.0698,42.3690
Resolving api.openstreetmap.org (api.openstreetmap.org)... 130.117.76.11, 130.117.76.12, 130.117.76.13, ...
Connecting to api.openstreetmap.org (api.openstreetmap.org)|130.117.76.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/xml]
Saving to: ‘kendall.osm’

kendall.osm             [             <=>    ]   6.42M  2.47MB/s    in 2.6s    

2021-11-29 22:10:30 (2.47 MB/s) - ‘kendall.osm’ saved [6734471]



In [ ]:
#importing public transit lines from OSM
! netconvert --osm-files /content/kendall.osm  --geometry.remove --ramps.guess --junctions.join --tls.guess-signals --tls.discard-simple --tls.join --tls.default-type actuated -o kendall.net.xml --remove-edges.isolated true --osm.stop-output.length 25 --ptstop-output kendallPTadditional.xml --ptline-output kendallPTlines.xml

In [ ]:
#converting kendall.osm to sumo readable file
#!python3 /usr/local/lib/python3.7/dist-packages/sumo/tools/osmBuild.py --osm-file kendall.osm --pedestrians -p "kendall" --netconvert-typemap /usr/local/lib/python3.7/dist-packages/sumo/data/typemap/osmNetconvert.typ.xml,/usr/local/lib/python3.7/dist-packages/sumo/data/typemap/osmNetconvertPedestrians.typ.xml,/usr/local/lib/python3.7/dist-packages/sumo/data/typemap/osmNetconvertBicycle.typ.xml

## Demand Modelling

### Senario I: Weekday Peak Hours


####Vehicles
generate random trips in the network with randomTrips.py

cars-weekday peak hours (morning) original

number of cars: Kendall Square Mobility Data Report:before Covid, peak hour average 8000 cars/hour at Roadway Cordon Locations https://storymaps.arcgis.com/stories/44ecfce9affc4dcca05007cce348eff2#ref-n-rUTU_o

max speed of car is set to 20 mph according to the [speed limit posted by the City of Cambridge](https://www.cambridgema.gov/StreetsAndTransportation/PoliciesOrdinancesAndPlans/VisionZero/SpeedLimitsInCambridge).

fringe factor: According to ["PTDM Survey Results: Kendall Square Area Employee Commute Trip Origins"](https://storymaps.arcgis.com/collections/2bdfc482220c46928d6e30de69dd1c2f?item=1)...Percent of Total Commuters Using SOV From Zip Code located in Kendall square is 11.33%. Therefore, the fringe factor can be estimated to be (1-11.33%)/11.33% = 8

In [ ]:
!python /usr/local/lib/python3.7/dist-packages/sumo/tools/randomTrips.py\
 -n kendall.net.xml\
 -b 0 -e 1200\
 -p 0.4\
 --binomial 20\
 --fringe-factor 8 -L\
 --vehicle-class passenger\
 --min-distance 1000\
 --trip-attributes="maxSpeed=\"20\""\
 --prefix "car"\
 -r peakVeh.rou.xml\
 -o peakVeh.trips.xml\
 --validate

calling /usr/local/lib/python3.7/dist-packages/sumo/bin/duarouter -n kendall.net.xml -r peakVeh.trips.xml --ignore-errors --begin 0 --end 1200 --no-step-log --no-warnings -o peakVeh.rou.xml
Success.
calling /usr/local/lib/python3.7/dist-packages/sumo/bin/duarouter -n kendall.net.xml -r peakVeh.trips.xml --ignore-errors --begin 0 --end 1200 --no-step-log --no-warnings -o peakVeh.trips.xml.tmp --write-trips
Success.


####Buses
bus-weekday peak hours original

number of buses: es enter into kendall square in 20 minutes? p=1200/x  (subsitute the number x with the answer)

max speed of car is set to 20 mph according to the [speed limit posted by the City of Cambridge](https://www.cambridgema.gov/StreetsAndTransportation/PoliciesOrdinancesAndPlans/VisionZero/SpeedLimitsInCambridge).

fringe factor: no buses depart within kendall square, therefore the fringe factor is set to 100.

bus frequency: https://cdn.mbta.com/sites/default/files/route_pdfs/2021-fall/068-04.21.2.0.pdf

In [ ]:
!python /usr/local/lib/python3.7/dist-packages/sumo/tools/ptlines2flows.py\
 -n kendall.net.xml\
 -b 0 -e 1200\
 -s kendallPTadditional.xml\
 -l kendallPTlines.xml\
 -p 1200\
 --use-osm-routes\
 --extend-to-fringe\
 --vtype-prefix "bus"\
 -o peakBus.rou.xml\
 -t peakBus.trips.xml

generating trips...
done.
running SUMO to determine actual departure times...
done.
creating routes...
done.


In [ ]:
#do not run this - expired version
# !python /usr/local/lib/python3.7/dist-packages/sumo/tools/randomTrips.py\
#  -n kendall.net.xml\
#  -b 0 -e 1200\
#  -p 10\
#  --binomial 5\
#  --fringe-factor 100 -L\
#  --vehicle-class bus\
#  --trip-attributes="maxSpeed=\"20\""\
#  --prefix "bus"\
#  -r peakBus.rou.xml\
#  -o peakBus.trips.xml\
#  --validate

calling /usr/local/lib/python3.7/dist-packages/sumo/bin/duarouter -n kendall.net.xml -r peakBus.trips.xml --ignore-errors --begin 0 --end 1200 --no-step-log --no-warnings -o peakBus.rou.xml
Success.
calling /usr/local/lib/python3.7/dist-packages/sumo/bin/duarouter -n kendall.net.xml -r peakBus.trips.xml --ignore-errors --begin 0 --end 1200 --no-step-log --no-warnings -o peakBus.trips.xml.tmp --write-trips
Success.


####Pedestrians

In [ ]:
#pedestrians
!python /usr/local/lib/python3.7/dist-packages/sumo/tools/randomTrips.py\
 -n kendall.net.xml\
  -b 0 -e 1200\
  -p 1\
  --binomial 20\
  --fringe-factor 0.6\
  --pedestrians --persontrips\
  --vehicle-class pedestrian\
  --trip-attributes="maxSpeed=\"4\""\
  --prefix "pedestrian"\
  -r ped.rou.xml\
  -o ped.trips.xml\
  --validate

# max speed of pedestrian is set to 4mph according to https://en.wikipedia.org/wiki/Walking#:~:text=Specific%20studies%20have%20found%20pedestrian,%2Fh%20(4.0%20mph).

calling /usr/local/lib/python3.7/dist-packages/sumo/bin/duarouter -n kendall.net.xml -r ped.trips.xml --ignore-errors --begin 0 --end 1200 --no-step-log --no-warnings -o ped.rou.xml
Success.
calling /usr/local/lib/python3.7/dist-packages/sumo/bin/duarouter -n kendall.net.xml -r ped.trips.xml --ignore-errors --begin 0 --end 1200 --no-step-log --no-warnings -o ped.trips.xml.tmp --write-trips
Success.


##Build Configuration

In [ ]:
#build sumo configuration
!sumo -n kendall.net.xml -r veh.trips.xml,bus.trips.xml,ped.trips.xml -C kendallTEST.sumocfg

##Outputs

In [ ]:
#run simulation and export fcd with geo coordinates

#fcd output
!sumo -c kendallTEST.sumocfg --fcd-output kendallFCD.xml --fcd-output.geo true

Loading configuration ... done.
Step #1300.00Warning: Collision of person 'pedestrian757' and person 'pedestrian1085', lane='34065677#2_0', time=1300.00.
Step #2620.00 (2ms ~= 500.00*RT, ~0.00UPS, vehicles TOT 553 ACT 0 BUF 0)                  


In [ ]:
!sumo -c kendallTEST.sumocfg --fcd-output kendallFCD_veh_only.xml --fcd-output.geo true --fcd-output.params vehicle

Loading configuration ... done.
Step #1300.00Warning: Collision of person 'pedestrian757' and person 'pedestrian1085', lane='34065677#2_0', time=1300.00.
Step #2620.00 (1ms ~= 1000.00*RT, ~0.00UPS, vehicles TOT 553 ACT 0 BUF 0)                 


In [ ]:
#Basic stats output
!sumo -c kendallTEST.sumocfg --statistic-output kendallStats.xml

Loading configuration ... done.
Step #1300.00Warning: Collision of person 'pedestrian757' and person 'pedestrian1085', lane='34065677#2_0', time=1300.00.
Step #2620.00 (1ms ~= 1000.00*RT, ~0.00UPS, vehicles TOT 553 ACT 0 BUF 0)                 


In [ ]:
#Tripinfo output
!sumo -c kendallTEST.sumocfg --tripinfo-output kendallTripInfoCO2.xml --device.emissions.probability 1
#The complete amount of CO2 emitted by the vehicle during the trip (unit:mg)

Loading configuration ... done.
Step #1300.00Warning: Collision of person 'pedestrian757' and person 'pedestrian1085', lane='34065677#2_0', time=1300.00.
Step #2620.00 (1ms ~= 1000.00*RT, ~0.00UPS, vehicles TOT 553 ACT 0 BUF 0)                 


In [ ]:
#Emission output
!sumo -c kendallTEST.sumocfg --emission-output kendallEmission.xml

Loading configuration ... done.
Step #1300.00Warning: Collision of person 'pedestrian757' and person 'pedestrian1085', lane='34065677#2_0', time=1300.00.
Step #2620.00 (1ms ~= 1000.00*RT, ~0.00UPS, vehicles TOT 553 ACT 0 BUF 0)                 


# 4. Analysis Module

Metrics:
1. Mobility
2. Socialability
3. Profitability
4. Sustainability

Scenarios:
**ADD NOTES**

## 4.1 Mobility

## 4.2 Socialability

## 4.3 Profitability

### 4.3.1 An Overview of Outdoor Dining in Cambridge

In [34]:
import json

outdoor_d = pd.read_json("https://data.cambridgema.gov/resource/qzsr-xrig.json")

display(outdoor_d)

fig1 = px.scatter_mapbox(outdoor_d, lat="latitude", lon="longitude", color="typepublicarea", hover_name="doingbusinessas", labels={"typepublicarea":"Outdoor Dining Option"}, color_discrete_map={"Not Applicable":"silver", "Assumed Sidewalk":"peachpuff", 
                                                                                                                                              "Parking Lane":"pink", "Sidewalk": "orange", "Other":"goldenrod", "Road": "salmon", "Parking Lane/Assumed Sidewalk":"magenta", 
                                                                                                                                              "Road + Sidewalk":"palevioletred", "Parking Lane + Sidewalk":"green", "Parking Lot": "teal"}, zoom=12, center={"lat":42.3669, "lon":-71.0749}, width=2000, height=1000)


# fig2 = px.pie(outdoor_d, names="doingbusinessas", values="typepublicarea", color="typepublicarea", hover_name="doingbusinessas", labels={"typepublicarea":"Outdoor Dining Option"}, color_discrete_map={"Not Applicable":"silver", "Assumed Sidewalk":"peachpuff", 
#                                                                                                                                               "Parking Lane":"pink", "Sidewalk": "orange", "Other":"goldenrod", "Road": "salmon", "Parking Lane/Assumed Sidewalk":"magenta", 
#                                                                                                                                               "Road + Sidewalk":"palevioletred", "Parking Lane + Sidewalk":"green", "Parking Lot": "teal"})
fig1.show()

,doingbusinessas,corporatename,fulladdress,latitude,longitude,neighborhood,areatype,typepublicarea,alcohol,website
0,Royal Sonesta Hotel,Sonesta International Hotels Corp.,"40 Land Blvd, Cambridge, MA 02142",42.3669,-71.0749,East Cambridge,Private,Not Applicable,True,{'url': 'https://www.sonesta.com/us/massachuse...
1,Hotel Veritas,"Veritas in the Square, LLC","1 Remington St, Cambridge, MA 02138",42.3711,-71.1138,Mid-Cambridge,Private,Not Applicable,True,{'url': 'https://www.thehotelveritas.com/'}
2,Hotel Marlowe and Bambara,"Cambridge Lodging, LLC","25 Land Blvd, Cambridge, MA 02141",42.3678,-71.0749,East Cambridge,Private,Not Applicable,True,{'url': 'https://www.bambara-cambridge.com/'}
3,Sheraton Commander,"Commander Properties, Inc.","16 Garden St, Cambridge, MA 02138",42.3773,-71.1232,West Cambridge,Private,Not Applicable,True,{'url': 'https://www.nubarcambridge.com/'}
4,AC Hotels by Marriott-Cambridge,"Colwen Management, Inc.","10 Acorn Park Dr, Cambridge, MA 02140",42.3987,-71.1473,North Cambridge,Private,Not Applicable,True,{'url': 'https://www.marriott.com/hotels/trave...
...,...,...,...,...,...,...,...,...,...,...
249,Muku,TTT&K Corp.,"411 Massachusetts Ave, Cambridge, MA 02139",42.3636,-71.1002,The Port,Private,Not Applicable,False,{'url': 'https://www.mukuramen.com/'}
250,Middlesex,Rush Hour Group LLC,"315 Massachusetts Ave, Cambridge, MA 02139",42.3625,-71.0985,The Port,Public,Assumed Sidewalk,True,{'url': 'http://www.middlesexlounge.us/'}
251,Cloud and Spirits,Love Art Udon Comm LLC,"795 Main St, Cambridge, MA 02139",42.3633,-71.0971,The Port,Public,Parking Lane,True,{'url': 'https://www.cloudandspirits.com/'}
252,Orale Paco's Mexican Grill,Orale Mexican Grill Inc,"847 Cambridge St, Cambridge, MA 02141",42.3725,-71.0909,Wellington-Harrington,Public,Parking Lane,False,{'url': 'http://www.oralemexicangrill.com/'}


## 4.4 Sustainability

# 5. Visualization

In [ ]:
#Set up


In [ ]:
#Parse XML output



tree1 = ET.parse('/content/kendallTripInfoCO2.xml')
root1 = tree1.getroot()

em_df_cols = ["ID", "TRAVEL_TIME", "CO", "CO2", "HC", "PMx", "NOx", ]
em_rows = []

for child in root1:
  if child.tag == 'tripinfo':
    travel_time = child.get('duration')
    id = child.get('id')
    co = float(child[0].get('CO_abs'))
    co2 = float(child[0].get('CO2_abs'))
    hc = child[0].get('HC_abs')
    pmx = child[0].get('PMx_abs')
    nox = child[0].get('NOx_abs')
    em_rows.append({"ID": id, "TRAVEL_TIME": travel_time, "CO": co, "CO2": co2, "HC": hc, "PMx": pmx, "NOx": nox})

em_out_df = pd.DataFrame(em_rows, columns = em_df_cols)

display(em_out_df)


,ID,TRAVEL_TIME,CO,CO2,HC,PMx,NOx
0,car1,39.00,3463.143065,1.950611e+05,20.437904,3.973731,83.125527
1,car25,27.00,1327.806989,6.828306e+04,7.627892,1.150454,27.871751
2,car0,89.00,7450.890388,2.952720e+05,40.923667,6.015166,126.337375
3,car18,57.00,2337.535155,1.410430e+05,13.935439,2.416111,57.546237
4,car3,96.00,9878.013248,2.759475e+05,51.493082,5.967471,120.812804
...,...,...,...,...,...,...,...
548,car420,1539.00,200504.580319,3.779003e+06,1001.602148,89.143443,1695.221184
549,car545,1374.00,185379.914318,3.356765e+06,922.882443,80.091573,1513.512267
550,car590,1324.00,172129.852464,3.160383e+06,857.917827,74.942012,1422.439051
551,bus96,1476.00,27469.515366,8.692967e+06,6490.684474,2764.336894,90309.377932


In [ ]:
# Parse Veh-only output

tree2 = ET.parse('/content/kendallFCD.xml')
root2 = tree2.getroot()

for child in root2:
  for person in child.findall('person'):
    child.remove(person)

tree2.write('/content/veh_only_output.xml')

In [ ]:
from parser6 import readParser, start_end_only

res = readParser(tree2)
res2 = start_end_only(res)

print(res2["car1"])

trip_df_cols = ["ID", "START_TIME", "START_X", "START_Y", "END_TIME", "END_X", "END_Y"]
trip_rows = []

for key in res2:
  start_time = float(res2[key][0])
  start_x = float(res2[key][1])
  start_y = float(res2[key][2])
  end_time = float(res2[key][3])
  end_x = float(res2[key][4])
  end_y = float(res2[key][5])
  
  
  trip_rows.append({"ID":key, "START_TIME": start_time, "START_X": start_x, "START_Y": start_y, "END_TIME": end_time, "END_X": end_x, "END_Y": end_y})

trip_out_df = pd.DataFrame(trip_rows, columns = trip_df_cols)

final_out_df = pd.merge(em_out_df, trip_out_df, on='ID')
display(final_out_df)

['4.00', '-71.080624', '42.367596', '42.00', '-71.081446', '42.367703']


,ID,TRAVEL_TIME,CO,CO2,HC,PMx,NOx,START_TIME,START_X,START_Y,END_TIME,END_X,END_Y
0,car1,39.00,3463.143065,1.950611e+05,20.437904,3.973731,83.125527,4.0,-71.080624,42.367596,42.0,-71.081446,42.367703
1,car25,27.00,1327.806989,6.828306e+04,7.627892,1.150454,27.871751,56.0,-71.079149,42.368120,82.0,-71.079696,42.366233
2,car0,89.00,7450.890388,2.952720e+05,40.923667,6.015166,126.337375,0.0,-71.079926,42.367848,88.0,-71.084105,42.367250
3,car18,57.00,2337.535155,1.410430e+05,13.935439,2.416111,57.546237,42.0,-71.079926,42.367848,98.0,-71.081482,42.371236
4,car3,96.00,9878.013248,2.759475e+05,51.493082,5.967471,120.812804,13.0,-71.093679,42.366093,108.0,-71.089891,42.364890
...,...,...,...,...,...,...,...,...,...,...,...,...,...
548,car420,1539.00,200504.580319,3.779003e+06,1001.602148,89.143443,1695.221184,903.0,-71.086109,42.360513,2441.0,-71.078503,42.361820
549,car545,1374.00,185379.914318,3.356765e+06,922.882443,80.091573,1513.512267,1095.0,-71.086282,42.361168,2468.0,-71.094790,42.368786
550,car590,1324.00,172129.852464,3.160383e+06,857.917827,74.942012,1422.439051,1180.0,-71.083966,42.360493,2503.0,-71.094411,42.367516
551,bus96,1476.00,27469.515366,8.692967e+06,6490.684474,2764.336894,90309.377932,1040.0,-71.096114,42.364826,2515.0,-71.069706,42.364361


In [ ]:
df = final_out_df
display(df)


,ID,TRAVEL_TIME,CO,CO2,HC,PMx,NOx,START_TIME,START_X,START_Y,END_TIME,END_X,END_Y
0,car1,39.00,3463.143065,1.950611e+05,20.437904,3.973731,83.125527,4.0,-71.080624,42.367596,42.0,-71.081446,42.367703
1,car25,27.00,1327.806989,6.828306e+04,7.627892,1.150454,27.871751,56.0,-71.079149,42.368120,82.0,-71.079696,42.366233
2,car0,89.00,7450.890388,2.952720e+05,40.923667,6.015166,126.337375,0.0,-71.079926,42.367848,88.0,-71.084105,42.367250
3,car18,57.00,2337.535155,1.410430e+05,13.935439,2.416111,57.546237,42.0,-71.079926,42.367848,98.0,-71.081482,42.371236
4,car3,96.00,9878.013248,2.759475e+05,51.493082,5.967471,120.812804,13.0,-71.093679,42.366093,108.0,-71.089891,42.364890
...,...,...,...,...,...,...,...,...,...,...,...,...,...
548,car420,1539.00,200504.580319,3.779003e+06,1001.602148,89.143443,1695.221184,903.0,-71.086109,42.360513,2441.0,-71.078503,42.361820
549,car545,1374.00,185379.914318,3.356765e+06,922.882443,80.091573,1513.512267,1095.0,-71.086282,42.361168,2468.0,-71.094790,42.368786
550,car590,1324.00,172129.852464,3.160383e+06,857.917827,74.942012,1422.439051,1180.0,-71.083966,42.360493,2503.0,-71.094411,42.367516
551,bus96,1476.00,27469.515366,8.692967e+06,6490.684474,2764.336894,90309.377932,1040.0,-71.096114,42.364826,2515.0,-71.069706,42.364361


In [ ]:

px.set_mapbox_access_token("pk.eyJ1IjoibGlhbXoiLCJhIjoiY2oxZnE5MHFhMDByeTJybzJrZjlvYTR4YyJ9.qwj-DfGjKw0dchnd5cNNDQ")

fig = ff.create_hexbin_mapbox(
    data_frame=df, lat="START_Y", lon="START_X",
    nx_hexagon=50, opacity=0.9, labels={"color": "Cumulated CO2 Emission"},
    color="CO2",agg_func=np.sum, color_continuous_scale="Magenta", height=400, min_count=1, show_original_data=True,
    original_data_marker=dict(size=4, opacity=0.6, color="deeppink"))
fig.show()

In [ ]:


px.set_mapbox_access_token("pk.eyJ1IjoibGlhbXoiLCJhIjoiY2oxZnE5MHFhMDByeTJybzJrZjlvYTR4YyJ9.qwj-DfGjKw0dchnd5cNNDQ")
df = px.data.carshare()

fig = create_hexbin_mapbox(
    data_frame=df, lat="centroid_lat", lon="centroid_lon",
    nx_hexagon=10, opacity=0.9, labels={"color": "Point Count"},
)
fig.update_layout(margin=dict(b=0, t=0, l=0, r=0))
fig.show()

TypeError: ignored

In [ ]:


# def line_array(df, cols):
#     return np.pad(df.loc[:,cols].values, [(0, 0), (0, 1)], constant_values=None).reshape(1,(len(df)*3))[0]
    
df = final_out_df

# SX = df['START_X']
# SY = df['START_Y']
# EX = df['END_X']
# EY = df['END_Y']

# fig = px.line_mapbox(lat=[SY, EY], 
#                      lon=[SX, EX],
# )

# fig.update_layout(mapbox_style="stamen-terrain",
#                   mapbox_zoom=4,
#                   mapbox_center_lon=df.loc[:,["START_X","END_X"]].mean().mean(),
#                   mapbox_center_lat=df.loc[:,["START_Y","END_Y"]].mean().mean(),
#                   margin={"r":0,"t":0,"l":0,"b":0}
#                  )

fig = go.Figure()

flight_paths = []
for i in range(len(df)):
    fig.add_trace(
        go.Scattergeo(
            locationmode = 'USA-states',
            lon = [df['START_X'][i], df['END_X'][i]],
            lat = [df['START_Y'][i], df['END_Y'][i]],
            mode = 'lines',
            line = dict(width = 1,color = 'red'),
        )
    )

fig.update_layout(
    showlegend = False,
    geo = dict(
        scope = 'north america',
        projection_type = 'azimuthal equal area',
        showland = True,
        landcolor = 'rgb(243, 243, 243)',
        countrycolor = 'rgb(204, 204, 204)',
    ),
)

fig.show()


In [ ]:
px.scatter(data_frame=df, x='TRAVEL_TIME', y='CO2', opacity=0.3)

In [ ]:
!pip install keplergl

     |████████████████████████████████| 9.7 MB 8.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.0 MB 42.4 MB/s 
     |████████████████████████████████| 6.3 MB 45.4 MB/s 
     |████████████████████████████████| 15.4 MB 40.2 MB/s 
  Created wheel for keplergl: filename=keplergl-0.3.2-py2.py3-none-any.whl size=17922343 sha256=c1f0564a203d93e2635365e430a889e73bbb8ea1d575838a8ff4295a8ec4f923
  Stored in directory: /root/.cache/pip/wheels/4b/19/e5/0852b7888ee350e9c57087ccc61eeab9daf50f5caac9c690ec
Successfully built keplergl


In [ ]:
from IPython.display import Javascript
display(Javascript('''
  google.colab.widgets.installCustomManager('https://ssl.gstatic.com/colaboratory-static/widgets/colab-cdn-widget-manager/6a14374f468a145a/manager.min.js');
'''))
from google.colab import output
output.enable_custom_widget_manager()

from keplergl import KeplerGl


map_1 = KeplerGl(height=800, data={"data1": df})
map_1



<IPython.core.display.Javascript object>

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'data1':          ID TRAVEL_TIME             CO  ... END_TIME       END_X      END_Y
0      car…